In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
# from langchain.prompts import PromptTemplate
# from langchain.chains import ConversationalRetrievalChain
# from langchain.memory import ConversationBufferMemory




# Load and Text Splitter 
Use langchain PyPDFLoader to load data in data file of astro_book_data.
Use Text Splitter to split pdf into chunks.

In [19]:
import os
path = "astro_book_data"
items = os.listdir(path)
documents = []
for filename in items:
    loader = PyPDFLoader(path + "/" + filename)
    documents.append(loader.load())

In [21]:
print(len(documents))
print(len(documents[0]))

4
145


In [28]:
texts = []
for document in documents:
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
    text = text_splitter.split_documents(document)
    for item in text:
        texts.append(item)
print(texts[0])

page_content='ASTROLOGY AS A NEW MODEL OF REALITY 
THE PHILOSOPHY OF ASTROLOGY AND ITS RELATION TO JUNGIAN 
PSYCHOLOGY 
  
By:Michael McMullin 
1993 
INTRODUCTION 
Part One : The Psychological Types. 
Chapter one: Astrology and the Psychological Types. 
Chapter Two: The Philosophy of the Types.
Part Two: Non-Aristotelian Logic. 
Chapter Three: Levels of Reality. 
Chapter Four: "Think in Other Categories".
Part Three: The Planetary Archetypes. 
Chapter Five: The Inner Planets. 
Chapter Six: Jupiter and Saturn.
Part Four: Beethoven. 
Beethoven: The Outer Planets and the Embodiment of an Archetype.
Postscript 
Appendix 
Synopsis 
  
PART ONE 
THE PSYCHOLOGICAL TYPES 
  
INTRODUCTION
We live in an extraordinarily compartmentalised society, in our thinking, a society of 
enclaves, and of innumerable culs-de-sac, all studiously disconnected with one another. It is a 
state of affairs resembling the fishing communities on the coast of Newfoundland, where life 
is quite primitive and climate a

In [33]:
embeddings = HuggingFaceEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings,persist_directory="./chromadb_store")  # store the embedding in docsearch using Chromadb
print('document ingested')

document ingested


In [34]:
# Save to disk
docsearch.persist()

In [35]:
# Load Chroma
embeddings = HuggingFaceEmbeddings()

# Load from disk
docsearch = Chroma(persist_directory="./chromadb_store", embedding_function=embeddings)

# Now you can use docsearch like before
results = docsearch.similarity_search("test document")
print(results)

[Document(metadata={'author': 'Isabel M. Hickey', 'creationdate': '2025-06-10T03:37:13+00:00', 'creator': '', 'moddate': '2025-06-10T03:37:18+00:00', 'page': 221, 'page_label': '222', 'producer': 'ConvertAPI', 'source': 'astro_book_data/Astrology_ A Cosmic Science_ The Classic Work on Spiritual -- Arroyo, Stephen; Hickey, Isabel M_ -- New revised & combined edition, Sebastopol, Calif, -- 9780916360634 -- f3a41ee362769cb9829a9316.pdf', 'title': 'Astrology, A Cosmic Science', 'total_pages': 445}, page_content="guard against nerve strain and tension. Test lies in learning to have faith and\noptimistic viewpoints in mental attitudes toward life. Inclined to be negative\nin approach with Saturn afflicted in this sign.\nSATURN IN CANCER (Detriment)\nEmotions too crystallized. May come, not from innate selfishness, but\nfrom a hurt to the feelings early in life or by a parent. The result would be\nthat he would build up a self-protective coating where emotions are\nconcerned. Can be kind but 

# 前面完成了

In [ ]:
from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes, DecodingMethods
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM
model_id = 'google/flan-ul2'
parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,  
    GenParams.MIN_NEW_TOKENS: 130, # this controls the minimum number of tokens in the generated output
    GenParams.MAX_NEW_TOKENS: 256,  # this controls the maximum number of tokens in the generated output
    GenParams.TEMPERATURE: 0.5 # this randomness or creativity of the model's responses
}
credentials = {
    "url": "https://us-south.ml.cloud.ibm.com"
}

project_id = "skills-network"
model = Model(
    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id
)
flan_ul2_llm = WatsonxLLM(model=model)

In [ ]:
from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai import HarmBlockThreshold, HarmCategory
import os

# 設定 API 金鑰 (建議使用環境變數)
os.environ["GOOGLE_API_KEY"] = "your-google-api-key"
# 或者直接在初始化時傳入: google_api_key="your-api-key"

# 模型 ID 對應
model_id = ''  # 對應原本的 'google/flan-ul2'

# 參數設定 (對應原本的 parameters 字典)
parameters = {
    "temperature": 0.5,        # 對應原本的 TEMPERATURE
    "max_output_tokens": 256,  # 對應原本的 MAX_NEW_TOKENS
    "top_p": 0.8,             # Google 特有參數
    "top_k": 40,              # Google 特有參數
}

# 安全設定 (Google API 特有)
safety_settings = {
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
}

# 創建 LangChain 相容的 LLM 實例 (完全對應原本的格式)
flan_ul2_llm = GoogleGenerativeAI(
    model=model_id,
    google_api_key=os.getenv("GOOGLE_API_KEY"),  # 可選，如果已設定環境變數
    temperature=parameters["temperature"],
    max_output_tokens=parameters["max_output_tokens"],
    top_p=parameters["top_p"],
    top_k=parameters["top_k"],
    safety_settings=safety_settings
)

In [ ]:
qa = RetrievalQA.from_chain_type(llm=flan_ul2_llm, 
                                 chain_type="stuff", 
                                 retriever=docsearch.as_retriever(), 
                                 return_source_documents=False)
query = "what is mobile policy?"
qa.invoke(query)